# 4.2 Spectral Graph Bipartitioning
<a target="_blank" href="https://colab.research.google.com/github/SaajanM/mat422-homework/blob/main/4.2%20Spectral%20Graph%20Bipartitioning/spectral_graph_bipart.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

In [ ]:
# Install a numpy package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install matplotlib

In [ ]:
# Import the numpy package
import numpy as np
import matplotlib
import matplotlib.pyplot as pyplt
from mpl_toolkits.mplot3d import Axes3D
import math
import matplotlib.pyplot as plt
from scipy import interpolate

$\newcommand\norm[1]{\left\lVert#1\right\rVert}$
$\newcommand\abs[1]{\left\lvert#1\right\rvert}$
$\newcommand\argmax{\text{arg}\,\text{max}}$
$\newcommand\argmin{\text{arg}\,\text{min}}$
$\newcommand\cut{\mathop{\text{cut}}}$

## Section 4.2 Spectral Graph Bipartitioning

In general, graph partitioning is a problem that aims to **cut** a graph into two subgraphs such that the number of edges "crossing the border" is minimized (or the sum of weights thereof). An additional constraint is that we wish to have the two subgraphs of close size. We formalize this operation of "cutting" below

**Definition:** For a weighted graph $G=(V,E)$, given a bipartition of $V$ into disjoint $V_1,V_2$, the **cut** between them is defined as
$$
\cut(V_1,V_2) = \sum_{i\in V_1}\sum_{j\in V_2} M_{ij}
$$
where $M$ is the adjacency matrix of $G$

The **classical graph bipartitioning problem** is to find a bipartition of $G$ into $V_1^\ast,V_2^\ast$ such that $\cut(V_1^\ast,V_2^\ast) = \min_{V_1,V_2} \cut(V_1,V_2)$

In order to solve this problem let us define the **partition vector** $\mathbf{p}$ to capture the partition:
$$
\mathbf{p}_i = \begin{cases}
+1, & i\in V_1\\
-1, & i\in V_2
\end{cases}
$$

In order to help calculate the value of the cut we can define the **Rayleigh Quotient** of a bipartition of a graph $G=(V,E)$ into $V_1, V_2$ described by partition vector $\mathbf{p}$:
$$
\frac{\mathbf{p}^T L \mathbf{p}}{\mathbf{p}^T \mathbf{p}}
$$
where $L$ is the Lapacian matrix of the graph $G$.

We find that the Rayleigh Quotient of a bipartition of a graph $G=(V,E)$ into $V_1, V_2$ described by partition vector $\mathbf{p}$ is equivalent to $\frac{4}{\abs{V}}\cut(V_1,V_2)$

Because we must consider the balance of the cut, we first define a diagonal matrix $W$ called the **weight matrix** that gives a precedence to each node. That is, $W_{ii}$ represents the weight of node $i$. We generalize this to partitions be defining $W_{V_l}=\sum_{i\in V_l} W_{ii}$.

Now we define our optimization criterion to be minimizing the following objective function:
$$
Q(V_1,V_2)=\frac{\cut(V_1,V_2)}{W_{V_1}} + \frac{\cut(V_1,V_2)}{W_{V_2}}
$$

Using a generalized form of the partition vector and the Rayleigh quotient, we find that we can minimize $Q$ by instead minimizing $\frac{\mathbf{q}^T L \mathbf{q}}{\mathbf{q}^T W \mathbf{q}}$, where $L$ is the laplacian matrix, $W$ is the weight matrix, and
$$
\mathbf{q}_i = \begin{cases}
+\sqrt{\frac{W_{V_2}}{W_{V_1}}}, & i\in V_1\\
-\sqrt{\frac{W_{V_1}}{W_{V_2}}}, & i\in V_2\\
\end{cases}.
$$

This new objective can be minimized by partitioning such that $\mathbf{q}$ is equivalent to the eigenvector corresponding to the second smallest eigenvalue of the generalized eigenvalue problem $L\mathbf{x}=\lambda W\mathbf{x}$

### Common Choices For $W_{ii}$

In so far, we have yet to actually pick any values for our weight matrix entries. Here we present two common methods:

First we have the **Ratio Cut** method, where all $W_{ii}=1$. This ascribes equal weights to all nodes and thus does not care much at all about the edge weights present in the adjacency matrix $M$ of the graph $G$.

Next, we have the **Normalized Cut** method, where $W_{ii} = \sum_k M_{ik}$. This means that the weight of a node is equivalent to the sum of the weights of the incident edges.

### Usage In Code

The closest implementation I could find was in `scikit-learn`'s Spectral clustering, with `n_clusters` set to 2 and `assign_labels` set to `'cluster_qr'`